In [5]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from pathlib import Path
import pandas as pd

In [6]:
# Load training data,
df = pd.read_csv('resource/train.csv')
df.shape
df.head()

,filename,category
0,45e2d0c97f7bdf8cbf3594beb6fdcda0.jpg,3
1,f74d1a5fc2498bbbfa045c74e3cc333e.jpg,3
2,f6c172096818c5fab10ecae722840798.jpg,3
3,251ffd610399ac00fea7709c642676ee.jpg,3
4,73c7328b8eda399199fdedec6e4badaf.jpg,3


In [7]:
#Image Paths
from pathlib import Path

pathlist = Path('resource/train').glob('**/*.jpg')
pathlistDict = {}
for path in pathlist:
     # because path is object not string
     path_in_str = str(path)
     imgName = path_in_str.rsplit('/',1)[1]
     pathlistDict[imgName] = path_in_str

df['path'] = df['filename'].map(pathlistDict)
minCategory = df['category'].min()
maxCategory = df['category'].max()
df.set_index('category')
sampled_df = pd.DataFrame(columns=['filename', 'category','path'])
for i in range(minCategory,maxCategory + 1):
     categorized = df[df['category'] == i]
     sampled_cat = categorized.sample(frac=0.15, random_state=1)
     sampled_df = sampled_df.append(sampled_cat)
sampled_df
# sampled_df = df.sample(frac=0.5, random_state=1)
# sampled_df.head()

,filename,category,path
53136,e850ac9ba4408f382e7babc32af12021.jpg,0,resource/train/train/00/e850ac9ba4408f382e7bab...
55392,7413808b024a44b1a94b0012af181135.jpg,0,resource/train/train/00/7413808b024a44b1a94b00...
55521,b4f1cb2f28a6212dff2d82f771579c5b.jpg,0,resource/train/train/00/b4f1cb2f28a6212dff2d82...
53386,c79da14c116b213c7f14882c912959f2.jpg,0,resource/train/train/00/c79da14c116b213c7f1488...
54879,ff528c0b3bf525fc28530c9dc4edf700.jpg,0,resource/train/train/00/ff528c0b3bf525fc28530c...
...,...,...,...
80923,7f93e6d91f9dcc9bb9f28a82a2f306a5.jpg,41,resource/train/train/41/7f93e6d91f9dcc9bb9f28a...
79299,8e227f4a0448a062eb8267e424c903d1.jpg,41,resource/train/train/41/8e227f4a0448a062eb8267...
81444,45df8842f798b46bde954cae8eb792b4.jpg,41,resource/train/train/41/45df8842f798b46bde954c...
79041,00c1dddee29a7f05874ac97b209514b0.jpg,41,resource/train/train/41/00c1dddee29a7f05874ac9...


In [8]:
imgArray = []
count = 0
for idx, row in sampled_df.iterrows():
    path = row['path']
    image = load_img(path, grayscale=True, target_size=(200, 200))
    # x = img_to_array(image)
    # imgArray.append(x)
    imgArray.append(image)
    if(count > 10000):
        break
    count = count + 1
len(imgArray)

10002

In [5]:
# MODEL
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 150, 150), data_format='channels_first'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])